<h2>Colab tutorial in 3 minutes!<h2>

In [ ]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/inN8seMm7UI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## Please run these cells

In [ ]:
! gdown --id 1fdCV3sin_ZZ9BhZt6BhquPtZ4IQ8UY96
! gdown --id 11yepXZ4geifWW1kJ34sHdF94tWl0Y8M3

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1fdCV3sin_ZZ9BhZt6BhquPtZ4IQ8UY96
To: /content/emails.rar
100% 1.13M/1.13M [00:00<00:00, 45.7MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=11yepXZ4geifWW1kJ34sHdF94tWl0Y8M3
To: /content/stop-words.txt
100% 14.3k/14.3k [00:00<00:00, 14.4MB/s]


In [ ]:
! apt install unrar

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unrar is already the newest version (1:5.5.8-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
! mkdir ./dataset

mkdir: cannot create directory ‘./dataset’: File exists


In [ ]:
! unrar x /content/emails.rar /content/dataset > /dev/null



Would you like to replace the existing file /content/dataset/spamtraining/spamtraining (1).txt
 21865 bytes, modified on 2015-10-22 21:20
with a new one
 21865 bytes, modified on 2015-10-22 21:20

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit q


In [ ]:
! pip install hazm -qq

###Import

In [ ]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from re import findall
from hazm import Stemmer, word_tokenize
from sklearn.feature_selection import chi2, SelectKBest
from glob import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

"""Countvectorzer initialize"""
vect = CountVectorizer()
is_file = True

"""stemmer from hazm"""
stemmer = Stemmer()
stopwords = set(open('stop-words.txt', encoding='utf8').read().splitlines())
hamtrains = glob('dataset/hamtraining/hamtraining*.txt')
spamtrains = glob('dataset/spamtraining/spamtraining*.txt')

hamtests = glob('dataset/hamtesting/hamtesting*.txt')
spamtests = glob('dataset/spamtesting/spamtesting*.txt')

ham_spam_test = hamtests + spamtests
ham_spam_train = hamtrains + spamtrains

y_train = [False for i in range(300)]
y_spam_train = [True for i in range(300)]
y_train.extend(y_spam_train)


y_test = [False for i in range(200)]
y_spam_test = [True for i in range(200)]
y_test.extend(y_spam_test)

### Preprocess

In [ ]:
class const:
    farsi = ('ب', 'س', 'ش', 'ل', 'ت', 'ن', 'م', 'گ', 'ظ', 'ط', 'ز',
             'ر', 'ژ', 'ذ', 'د', 'پ', 'چ', 'ج', 'ح', 'ع', 
             'خ', 'غ', 'ف', 'ق', 'ث', 'ص', 'ض','\u0020', '\u060c','؟', '!', '?', '.', ':','\n','_')

    alef = ('ا', 'آ', 'ء', 'أ', 'إ')
    vav = ('و', 'ؤ')
    heh = ('ه', 'ة', 'ە')
    yah = ('ی', 'ي', 'ئ', 'ى')
    kaf = ('ک', 'ك')
    punc = ('_', '-')

def persian_char(char):
    if char in const.farsi:
        return char
    if char in const.alef:
        return const.alef[0]
    if char in const.vav:
        return const.vav[0]
    if char in const.heh:
        return const.heh[0]
    if char in const.yah:
        return const.yah[0]
    if char in const.kaf:
        return const.kaf[0]
    if char in const.punc:
      return ' '
    return ''

def pre_process(path):
    if is_file == False:
        text = path
    else:
        text = open(path, encoding='utf8').read()

#     urls = len(findall('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', text))
    
    map_test = map(persian_char, text)
    sentence = ''.join(map_test)
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if w not in stopwords] #+ ['url' for i in range(urls)]
    filtered_sentence = ' '.join(filtered_sentence)
    return filtered_sentence

In [ ]:
def feature(data):
    if is_file == False:
        feature_list = [pre_process(data)]
    else:    
        map_loop = map(pre_process, data)
        feature_list = list(map_loop)
    return feature_list

In [ ]:
def vectorize(feature_list):
    X_dtm = vect.fit_transform(feature_list)
    X_dtm = X_dtm.toarray()
    return X_dtm

In [ ]:
def feature_selection(k,X_dtm):
    chi2_features = SelectKBest(chi2, k=k)
    X_kbest_features = chi2_features.fit_transform(X_dtm, y_train)
    return X_kbest_features, chi2_features

In [ ]:
def transform(data,chi2_features):
    tokens = feature(data)
    x0 = vect.transform(tokens).toarray()
    cx = chi2_features.transform(x0)
    return cx

In [ ]:
X_dtm = vectorize(feature(ham_spam_train))
x_train, chi2_features = feature_selection(700,X_dtm)

### Just for Intuition

In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train,y_train)

MultinomialNB()

In [ ]:
"""Just for test and compare"""
x_test = transform(ham_spam_test, chi2_features)
predict_val = mnb.predict(x_test)

In [ ]:
"""Show metrics and score!"""
print(classification_report(y_test, predict_val))

              precision    recall  f1-score   support

       False       0.93      0.97      0.95       200
        True       0.97      0.93      0.95       200

    accuracy                           0.95       400
   macro avg       0.95      0.95      0.95       400
weighted avg       0.95      0.95      0.95       400



## Naive bayes model

<h2 dir="rtl"> ویدیو بسیار مفید برای ساخت مدل<h2>
<h2><a href="https://www.youtube.com/watch?v=O2L2Uv9pdDA">Naive Bayes, Clearly Explained!!!</a><h2>

<h3 dir="rtl">
متغیر‌هایی که در مرحله تست و ترین مدل مورد استفاده قرار میگیرند.

x_train, x_test, y_train, y_test

خرجی تابع
transform 
وکتورایز شده هر یک از سمپل‌های موجود در دیتاست تست است.
به طوری که تعداد اعضای 
test_data
برابر با 400 نمونه متنی است که برای تست کردن مدل مورد استفاده قرار میگیرد.
هر یک از سمپل‌ها دارای وکتوری به طول 700 میباشد، که تعداد فیچرهای(کلمات) برتر انتخاب شده توسط تابع SelectKBest 
اند.

در این وکتور 700 بعدی مقدار هر یک از اندیس‌ها نشان دهنده تکرار آن کلمه در آن سمپل خاص هست.

راهنمایی ساخت جدول احتمالات:
شما با کمک تکرار کلمات در هر سمپل و اینکه هر کلمه در کل نمونه‌های 
spam
یا
ham
چند بار تکرار شده است میتوانید احتمال رخدادن هر کلمه را در هر دسته بدست آورید
که در نهایت با ضرب احتمالات بدست آمده تعیین میکنید که ایمیل 
spam
است یا نه.

به این شکل که بعد از ضرب احتمالات در هم دیگر؛ اگر مقدار احتمال بدست آمده با توجه به جدول احتمالاتی 
spam 
بیشتر از جدول 
ham(nonspam)
بود،
آنگاه آن ایمیل spam 
است.
<h3>

In [ ]:
test_data = transform(ham_spam_test, chi2_features)

In [ ]:
test_data[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,

<h3 dir="rtl"> 
شما باید با کمک این فیچر‌ها مدل 
naive bayes 
را برای تشخیص ایمیل‌های 
spam, nonspam
آماده سازی کنید.
خروجی کار شما باید لیبل‌های باشد که تعیین میکند،
ایمیل 
spam هست
یا نه.

<h3>

<h1>Please implement your naive bayse model in here.<h1>

In [ ]:
### START CODE HERE ###
#Build your naive bayes classifier!

spam_sum = np.zeros(700)
ham_sum = np.zeros(700)
n = len(x_train)

for i in range(n):
    if y_train[i] == True:
        spam_sum += x_train[i]
    else:
        ham_sum += x_train[i]

spam_word = 0 
ham_word = 0

for i in range (len(spam_sum)):
  spam_sum[i] += 1
  spam_word += spam_sum[i]

for i in range (len(ham_sum)):
  ham_sum[i] += 1
  ham_word += ham_sum[i]

conditional_prob_spam = spam_sum / spam_word
conditional_prob_ham = ham_sum / ham_word

def naive_bayes(email):  
  spam_count = 0 
  ham_count = 0

  for i in range (n):
    if y_train[i] == True:
      spam_count += 1
    else:
      ham_count += 1

  ham = ham_count/n  
  spam =spam_count/n 

  for i in range(700):
    if email[i]>0:
      ham = ham * conditional_prob_ham[i]
      spam = spam * conditional_prob_spam[i]

  if ham >= spam:
    return False
  else:
    return True

### END CODE HERE ###

In [ ]:
### START CODE HERE ###
# y_predict = YOUR_MODEL(test_data)

y_predict = np.zeros(len(test_data))

for i in range(len(test_data)):
  y_predict[i] = naive_bayes(test_data[i])



### END CODE HERE ###

### Result

In [ ]:
"""Show metrics and score!"""
print(classification_report(y_test, y_predict))
print('confusion matrix : ')
print(confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

       False       0.91      0.97      0.94       200
        True       0.97      0.91      0.94       200

    accuracy                           0.94       400
   macro avg       0.94      0.94      0.94       400
weighted avg       0.94      0.94      0.94       400

confusion matrix : 
[[195   5]
 [ 19 181]]
